In [114]:
import numpy as np
import pandas as pd

# importing the requests library
import requests

In [115]:
## Resources ##
# [1] https://www.arcgis.com/home/item.html?id=124c2187da8c41c59bde04fa67eb2872
# [2] https://data-wake.opendata.arcgis.com/datasets/Wake::restaurants-in-wake-county/api

In [184]:
## Sourcing Data ##

def getRestaurantsDf():
        # Source restaurants data in wake county from wakegov
        base_url = "https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/0/query?f=json&where=(FACILITYTYPE%20IN%20('Restaurant'))&outFields=*"

        # keep pagination in mind (maxRecordCount = 1000, so grabbing 1000 at a time)
        reqs = []
        reqs.append(base_url)
        reqs.append(base_url+'&resultOffset=1000&resultRecordCount=1000')
        reqs.append(base_url+'&resultOffset=2000&resultRecordCount=1000')
        reqs.append(base_url+'&resultOffset=3000&resultRecordCount=1000')

        # Sending get request and saving the response as response object
        # extracting data in json 
        rows = []
        for val in reqs:
            r = requests.get(url = val)
            data = r.json()['features']
            for d in data:
                rows.append(d['attributes'])
        return pd.DataFrame(rows)


df = getRestaurantsDf()

In [185]:
# Validate shape & data
# https://data-wake.opendata.arcgis.com/datasets/Wake::restaurants-in-wake-county/explore?filters=eyJGQUNJTElUWVRZUEUiOlsiUmVzdGF1cmFudCJdfQ%3D%3D&location=35.794181%2C-78.605600%2C3.98&showTable=true
print(df.shape)
print(df['OBJECTID'].nunique())
print(df['HSISID'].nunique())
display(df)

(2388, 15)
1975
1975


,OBJECTID,HSISID,NAME,ADDRESS1,ADDRESS2,CITY,STATE,POSTALCODE,PHONENUMBER,RESTAURANTOPENDATE,FACILITYTYPE,PERMITID,X,Y,GEOCODESTATUS
0,1838439,04092017585,MCALISTER'S DELI #100958,1061 PINE PLAZA DR,None,APEX,NC,27523,(919) 362-6518,1493251200000,Restaurant,19935,-78.829393,35.746571,M
1,1838442,04092018454,Deja Brew,5001 FALLS OF NEUSE RD,None,RALEIGH,NC,27609,(919) 239-4612,1601510400000,Restaurant,19953,-78.615575,35.849783,M
2,1838443,04092016734,GROCERY BOY JR.,1912 W GARNER RD,None,GARNER,NC,27529,(919) 772-5018,1408665600000,Restaurant,19956,-78.625807,35.725984,M
3,1838444,04092018583,Wendys,2007 NW CARY PKY,None,MORRISVILLE,NC,27560-5908,(919) 465-1979,1616544000000,Restaurant,19958,-78.812350,35.805386,M
4,1838445,04092015676,ROADRUNNERS PIZZA,8420-120 LOUISBURG RD,None,RALEIGH,NC,27616,(919) 266-7606,1279843200000,Restaurant,19951,-78.510921,35.894136,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383,1839045,04092018555,Lorraine's Coffee House,101 TIMBER POINTE LN,None,GARNER,NC,27529,(919) 714-7990,1615334400000,Restaurant,23334,-78.635795,35.693792,M
2384,1839046,04092018410,MOD Pizza - Millbrook Rd,2071 W Millbrook Rd,None,RALEIGH,NC,27612,(919) 987-3668,1595808000000,Restaurant,23336,-78.679714,35.857372,M
2385,1839047,04092017544,FIRST WATCH BRADFORD,1325 BRADFORD VIEW DR,None,CARY,NC,27513,(919) 655-8005,1488758400000,Restaurant,23345,-78.845692,35.792510,M
2386,1839050,04092016504,Comfort Inn,1001 WAKE TOWNE DR,None,RALEIGH,NC,27609,(919) 828-1813,1378339200000,Restaurant,23365,-78.624378,35.824434,M
